In [ ]:
THRESHOLD = 5

In [ ]:
%run database.ipynb
%run pathutils.ipynb
%run export.ipynb

In [ ]:
# Load and preview the data
connection = connect()
genre_stats_df = load_genre_statistics(connection)
display(genre_stats_df)

## Artist Statistics

In [ ]:
# Total artists
total_artists = genre_stats_df["Artists"].sum()

# Percentage of artists by genre
artist_pct = (genre_stats_df.set_index("Genre")["Artists"] / total_artists) * 100

# Split into large and small genres
large_artist_se = artist_pct[artist_pct >= THRESHOLD]
small_artist_se = artist_pct[artist_pct < THRESHOLD]

# Combine small genres into "Other"
if not small_artist_se.empty:
    large_artist_se["Other"] = small_artist_se.sum()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.pie(
    large_artist_se,
    labels=large_artist_se.index,
    autopct="%1.1f%%",
    startangle=90
)
plt.title("% of Artists by Genre")
plt.axis("equal")

# Export the chart
export_chart("artists-by-genre", "", "png")

plt.show()

## Album Statistics

In [ ]:
# Total albums
total_albums = genre_stats_df["Albums"].sum()

# Percentage of albums by genre
album_pct = (genre_stats_df.set_index("Genre")["Albums"] / total_albums) * 100

# Split into large and small genres
large_album_se = album_pct[album_pct >= THRESHOLD]
small_album_se = album_pct[album_pct < THRESHOLD]

# Combine small genres into "Other"
if not small_album_se.empty:
    large_album_se["Other"] = small_album_se.sum()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.pie(
    large_album_se,
    labels=large_album_se.index,
    autopct="%1.1f%%",
    startangle=90
)
plt.title("% of Albums by Genre")
plt.axis("equal")

# Export the chart
export_chart("albums-by-genre", "", "png")

plt.show()

# Track Statistics

In [ ]:
# Total tracks
total_tracks = genre_stats_df["Tracks"].sum()

# Percentage of tracks by genre
track_pct = (genre_stats_df.set_index("Genre")["Tracks"] / total_tracks) * 100

# Split into large and small genres
large_track_se = track_pct[track_pct >= THRESHOLD]
small_track_se = track_pct[track_pct < THRESHOLD]

# Combine small genres into "Other"
if not small_track_se.empty:
    large_track_se["Other"] = small_track_se.sum()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.pie(
    large_track_se,
    labels=large_track_se.index,
    autopct="%1.1f%%",
    startangle=90
)
plt.title("% of Tracks by Genre")
plt.axis("equal")

# Export the chart
export_chart("tracks-by-genre", "", "png")

plt.show()

## Spend Statistics

In [ ]:
# Total spends
total_spend = genre_stats_df["Spend"].sum()

# Percentage of spends by genre
spend_pct = (genre_stats_df.set_index("Genre")["Spend"] / total_spend) * 100

# Split into large and small genres
large_spend_se = spend_pct[spend_pct >= THRESHOLD]
small_spend_se = spend_pct[spend_pct < THRESHOLD]

# Combine small genres into "Other"
if not small_spend_se.empty:
    large_spend_se["Other"] = small_spend_se.sum()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.pie(
    large_spend_se,
    labels=large_spend_se.index,
    autopct="%1.1f%%",
    startangle=90
)
plt.title("% of Spend by Genre")
plt.axis("equal")

# Export the chart
export_chart("spend-by-genre", "", "png")

plt.show()

## Data Export

In [ ]:
import pandas as pd

export_to_spreadsheet(f"genre-statistics", {
    "Data": genre_stats_df,
    "Artist Statistics": large_artist_se,
    "Album Statistics": large_album_se,
    "Track Statistics": large_track_se,
    "Spend Statistics": large_spend_se
})